In [3]:
#Tweets 2 - Immigrants

In [2]:
#Import libraries
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import plotly.offline as py
#For this library it was required to install plotly and geopandas via Conda, also update setuptools and install ez_setup
import plotly.figure_factory as ff  
import vincent
from geopy.geocoders import Nominatim
import xlrd

#Configuration files
from config import gkey
py.offline.init_notebook_mode(connected=True)

In [2]:
tweets = pd.ExcelFile("Resources/TweetData/2nd_Search_Twitter_Posts.xls")
df = tweets.parse("searchd")
df['Longitude'] = ''
df['Latitude'] = ''
df.head(5)

,GUID,Date (EST),URL,Contents,Author,Name,Country,State/Region,City/Urban Area,Category,...,Following,Post Title,Post Type,Image URL,Object,Scene,Action,Brand,Longitude,Latitude
0,458611179154178048,2014-04-22 10:20:00,http://twitter.com/aretana/status/458611179154...,"RT @NateSilver538 Contrary to stereotypes, man...",@aretana,Alberto Retana,United States of America,California,Los Angeles,Negative,...,226,"RT @NateSilver538 Contrary to stereotypes, man...",Retweet,NaN,NaN,NaN,NaN,NaN,,
1,404753821881999360,2013-11-24 18:30:05,http://twitter.com/neecoley_smit/status/404753...,Cause i got so lost in the Spirit I think neig...,@neecoley_smit,Nicole Smit⚓️,United States of America,NaN,NaN,Negative,...,201,Cause i got so lost in the Spirit I think neig...,Tweet,NaN,NaN,NaN,NaN,NaN,,
2,460214337903132672,2014-04-26 20:30:23,http://twitter.com/cummins_lisa/status/4602143...,@Liberty92625 @jimhaycock as a Naturalized cit...,@cummins_lisa,Lisa Cummins,United States of America,Utah,NaN,Neutral,...,727,@Liberty92625 @jimhaycock as a Naturalized cit...,Reply,NaN,NaN,NaN,NaN,NaN,,
3,447701413268054016,2014-03-23 07:48:29,http://twitter.com/DoctorsCouncil/status/44770...,@DoctorsCouncil: Tell the House to Pass Immigr...,@DoctorsCouncil,Doctors Council SEIU,United States of America,NaN,NaN,Neutral,...,601,@DoctorsCouncil: Tell the House to Pass Immigr...,Tweet,NaN,NaN,NaN,NaN,NaN,,
4,580807084977045505,2015-03-25 15:02:54,http://twitter.com/bhamdailynews/status/580807...,A new visa program for entrepreneurs. Find out...,@bhamdailynews,Bham Daily News,United States of America,Alabama,Birmingham,Neutral,...,594,A new visa program for entrepreneurs. Find out...,Tweet,NaN,NaN,NaN,NaN,NaN,,


In [3]:
#Variables to be used during data collection
processed = 0
notfound = 0
count = 0

#Loop to collect Lat/Long for Agency Names
for (idx, row) in df.iterrows():
    try:
        #Store current row value for each column
        city = (row.loc['City/Urban Area'])
        state = (row.loc['State/Region'])
        #URL from Google APIs where Lat/Long values will be collected
        query_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={city},{state}&key={gkey}"
        #JSON request
        response = requests.get(query_url).json()
        #Variables to store lat/long values 
        lat = response["results"][0]["geometry"]["location"]["lat"]
        lng = response["results"][0]["geometry"]["location"]["lng"]
        #Lat/Long columns created and respective values stored in current row
        df.at[idx, 'Latitude'] = lat
        df.at[idx, 'Longitude'] = lng
        #Print to verify data is processing as expected
        print(f"{city},{state}:{lat},{lng}")
        #Count to validate how many city names have been processed
        processed += 1
        #Error handling if some of the city names are not found
    except:
        #Print to verify when an city is not found
        print("City not found")
        #Count to validate how many city names were not found
        notfound += 1
        continue
#Print to validate final count for both processed and not found city names
print(f"{processed} cities were processed. {notfound} cities were not found.")


Los Angeles,California:34.0522342,-118.2436849
nan,nan:36.103234,-95.87159059999999
nan,Utah:40.6867104,-111.8850049
nan,nan:36.103234,-95.87159059999999
Birmingham,Alabama:33.5206608,-86.80248999999999
nan,California:38.257725,-122.0666346
nan,nan:36.103234,-95.87159059999999
Washington, D.C.,District of Columbia:38.9071923,-77.0368707
San Francisco,California:37.7749295,-122.4194155
nan,nan:36.103234,-95.87159059999999
Beaufort,South Carolina:32.4315813,-80.6698286
nan,nan:36.103234,-95.87159059999999
Wilmington,Delaware:39.7390721,-75.5397878
nan,Michigan:39.94414099999999,-86.2374366
New York,New York:36.1023715,-115.1745559
nan,nan:36.103234,-95.87159059999999
Denver,Colorado:39.7392358,-104.990251
Irvine,California:33.6845673,-117.8265049
Tampa,Florida:27.950575,-82.4571776
nan,nan:36.103234,-95.87159059999999
New York,New York:36.1023715,-115.1745559
nan,New York:40.8205487,-73.9366757
nan,nan:36.103234,-95.87159059999999
nan,Pennsylvania:39.9552012,-75.1568535
nan,nan:36.103234

In [3]:
#df.to_csv('resources/TweetData/Tweets2_wCoords.csv')
FIPS_df = pd.read_csv('/Users/rorr/PythonStuff/Team_5/Resources/TweetData/Tweets2_wCoords.csv')
FIPS_df['FIPS'] = ''
FIPS_df['County Name'] = ''
FIPS_df['FIPS_block'] = ''

#URL to get FIPS code
url = 'https://geo.fcc.gov/api/census/block/find?'
#Variables to be used during data collection
processed = 0
notfound = 0

#Loop to add FIPS code and County name to Dataframe
for (idx, row) in FIPS_df.iterrows():
    try:
        #Store current row value for each column
        latitude = (row.loc['Latitude'])
        longitude = (row.loc['Longitude'])
        #URL to collect FIPS code and county name
        query_url = f"{url}&latitude={latitude}&longitude={longitude}&format=json"
        #JSON Request
        code = requests.get(query_url).json()
        #Variables to store required values (FIPS codes are converted to strings to keep leading zeros in the code)
        county_fips = str(code["County"]["FIPS"]).zfill(5)
        county_name = code["County"]["name"]
        block_fips = str(code["Block"]["FIPS"]).zfill(15)
        #New columns created and values collected from JSON stored in current row 
        FIPS_df.at[idx, 'FIPS'] = str(county_fips)
        FIPS_df.at[idx, 'County Name'] = county_name
        FIPS_df.at[idx, 'FIPS_block'] = str(block_fips)         
        #Count to validate how many records were processed
        processed += 1
        #Print to verify that information is processed
        print(f"{county_name}:{county_fips}")
    #Error handling if a location is not found
    except:
        #Print to validate if a location is not found
        print("Location Not Found")
        #Count to validate how many records were not found
        notfound += 1
        pass
#Print to validate final count for both processed and not found locations
print(f"{processed} locations were processed. {notfound} locations were not found.")
# Keep leading zeros


Los Angeles:06037
Tulsa:40143
Salt Lake:49035
Tulsa:40143
Jefferson:01073
Solano:06095
Tulsa:40143
District of Columbia:11001
San Francisco:06075
Tulsa:40143
Beaufort:45013
Tulsa:40143
New Castle:10003
Hamilton:18057
Clark:32003
Tulsa:40143
Denver:08031
Orange:06059
Hillsborough:12057
Tulsa:40143
Clark:32003
New York:36061
Tulsa:40143
Philadelphia:42101
Tulsa:40143
Tulsa:40143
Wake:37183
San Diego:06073
Collin:48085
District of Columbia:11001
Tulsa:40143
Tulsa:40143
Clark:32003
Dodge:31053
Orange:12095
Tulsa:40143
District of Columbia:11001
Hillsborough:12057
Clark:32003
Tulsa:40143
Tulsa:40143
San Francisco:06075
Los Angeles:06037
Philadelphia:42101
Los Angeles:06037
Tulsa:40143
Tulsa:40143
Tulsa:40143
Philadelphia:42101
Los Angeles:06037
Monmouth:34025
Collin:48085
Door:55029
District of Columbia:11001
Guilford:37081
Solano:06095
Harris:48201
Davidson:47037
Tulsa:40143
Collin:48085
Tulsa:40143
Los Angeles:06037
Monmouth:34025
Tulsa:40143
Tulsa:40143
District of Columbia:11001
Tulsa:4

,Unnamed: 0,GUID,Date (EST),URL,Contents,Author,Name,Country,State/Region,City/Urban Area,...,Image URL,Object,Scene,Action,Brand,Longitude,Latitude,FIPS,County Name,FIPS_block
0,0,458611179154178048,2014-04-22 10:20:00,http://twitter.com/aretana/status/458611179154...,"RT @NateSilver538 Contrary to stereotypes, man...",@aretana,Alberto Retana,United States of America,California,Los Angeles,...,NaN,NaN,NaN,NaN,NaN,-118.243685,34.052234,06037,Los Angeles,060372074001033
1,1,404753821881999360,2013-11-24 18:30:05,http://twitter.com/neecoley_smit/status/404753...,Cause i got so lost in the Spirit I think neig...,@neecoley_smit,Nicole Smit⚓️,United States of America,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-95.871591,36.103234,40143,Tulsa,401430069061000
2,2,460214337903132672,2014-04-26 20:30:23,http://twitter.com/cummins_lisa/status/4602143...,@Liberty92625 @jimhaycock as a Naturalized cit...,@cummins_lisa,Lisa Cummins,United States of America,Utah,NaN,...,NaN,NaN,NaN,NaN,NaN,-111.885005,40.686710,49035,Salt Lake,490351119061001
3,3,447701413268054016,2014-03-23 07:48:29,http://twitter.com/DoctorsCouncil/status/44770...,@DoctorsCouncil: Tell the House to Pass Immigr...,@DoctorsCouncil,Doctors Council SEIU,United States of America,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-95.871591,36.103234,40143,Tulsa,401430069061000
4,4,580807084977045505,2015-03-25 15:02:54,http://twitter.com/bhamdailynews/status/580807...,A new visa program for entrepreneurs. Find out...,@bhamdailynews,Bham Daily News,United States of America,Alabama,Birmingham,...,NaN,NaN,NaN,NaN,NaN,-86.802490,33.520661,01073,Jefferson,010730027001097


In [7]:
FIPS_df['FIPS'] = FIPS_df['FIPS'].apply(str)
FIPS_df['FIPS_block'] = FIPS_df['FIPS_block'].apply(str)
FIPS_df.to_csv('resources/TweetData/Tweets2_Final.csv')
FIPS_df.head()

,Unnamed: 0,GUID,Date (EST),URL,Contents,Author,Name,Country,State/Region,City/Urban Area,...,Image URL,Object,Scene,Action,Brand,Longitude,Latitude,FIPS,County Name,FIPS_block
0,0,458611179154178048,2014-04-22 10:20:00,http://twitter.com/aretana/status/458611179154...,"RT @NateSilver538 Contrary to stereotypes, man...",@aretana,Alberto Retana,United States of America,California,Los Angeles,...,NaN,NaN,NaN,NaN,NaN,-118.243685,34.052234,06037,Los Angeles,060372074001033
1,1,404753821881999360,2013-11-24 18:30:05,http://twitter.com/neecoley_smit/status/404753...,Cause i got so lost in the Spirit I think neig...,@neecoley_smit,Nicole Smit⚓️,United States of America,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-95.871591,36.103234,40143,Tulsa,401430069061000
2,2,460214337903132672,2014-04-26 20:30:23,http://twitter.com/cummins_lisa/status/4602143...,@Liberty92625 @jimhaycock as a Naturalized cit...,@cummins_lisa,Lisa Cummins,United States of America,Utah,NaN,...,NaN,NaN,NaN,NaN,NaN,-111.885005,40.686710,49035,Salt Lake,490351119061001
3,3,447701413268054016,2014-03-23 07:48:29,http://twitter.com/DoctorsCouncil/status/44770...,@DoctorsCouncil: Tell the House to Pass Immigr...,@DoctorsCouncil,Doctors Council SEIU,United States of America,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-95.871591,36.103234,40143,Tulsa,401430069061000
4,4,580807084977045505,2015-03-25 15:02:54,http://twitter.com/bhamdailynews/status/580807...,A new visa program for entrepreneurs. Find out...,@bhamdailynews,Bham Daily News,United States of America,Alabama,Birmingham,...,NaN,NaN,NaN,NaN,NaN,-86.802490,33.520661,01073,Jefferson,010730027001097


In [8]:
dtype_dic= {'FIPS': str, 
            'FIPS_block' : str}

immigration_df = pd.read_csv('/Users/rorr/PythonStuff/Team_5/Resources/TweetData/Tweets2_Final.csv',
                             dtype = dtype_dic )
immigration_df['FIPS'] = immigration_df['FIPS'].apply(str)

#Loop to keep leading zeros in FIPS codes
for (idx, row) in immigration_df.iterrows():
    fips_z = (row.loc['FIPS'])
    immigration_df.at[idx, 'FIPS'] = str(fips_z).zfill(5)

# Drop Non-Starters
print(immigration_df.count())
immigration_df = immigration_df[pd.notnull(immigration_df['State/Region'])]
print(immigration_df.count())

immigration_df['FIPS'] = immigration_df['FIPS'].apply(str)
immigration_df = immigration_df.groupby(['FIPS','County Name','State/Region'])[["Contents"]].count()
# #Remove FIPS, County Name and State columns from index
immigration_df = immigration_df.reset_index()

immigration_df.head()



Unnamed: 0         10000
Unnamed: 0.1       10000
GUID               10000
Date (EST)         10000
URL                10000
Contents           10000
Author             10000
Name                9994
Country            10000
State/Region        6513
City/Urban Area     4365
Category           10000
Emotion            10000
Source             10000
Klout Score         9602
Gender              5879
Posts              10000
Followers          10000
Following          10000
Post Title         10000
Post Type          10000
Image URL              0
Object                 0
Scene                  0
Action                 0
Brand                  0
Longitude           9990
Latitude            9990
FIPS               10000
County Name         9990
FIPS_block          9990
dtype: int64
Unnamed: 0         6513
Unnamed: 0.1       6513
GUID               6513
Date (EST)         6513
URL                6513
Contents           6513
Author             6513
Name               6511
Country            6

,index,Unnamed: 0,Unnamed: 0.1,GUID,Date (EST),URL,Contents,Author,Name,Country,...,Image URL,Object,Scene,Action,Brand,Longitude,Latitude,FIPS,County Name,FIPS_block
0,0,0,0,458611179154178048,2014-04-22 10:20:00,http://twitter.com/aretana/status/458611179154...,"RT @NateSilver538 Contrary to stereotypes, man...",@aretana,Alberto Retana,United States of America,...,NaN,NaN,NaN,NaN,NaN,-118.243685,34.052234,06037,Los Angeles,060372074001033
1,2,2,2,460214337903132672,2014-04-26 20:30:23,http://twitter.com/cummins_lisa/status/4602143...,@Liberty92625 @jimhaycock as a Naturalized cit...,@cummins_lisa,Lisa Cummins,United States of America,...,NaN,NaN,NaN,NaN,NaN,-111.885005,40.686710,49035,Salt Lake,490351119061001
2,4,4,4,580807084977045505,2015-03-25 15:02:54,http://twitter.com/bhamdailynews/status/580807...,A new visa program for entrepreneurs. Find out...,@bhamdailynews,Bham Daily News,United States of America,...,NaN,NaN,NaN,NaN,NaN,-86.802490,33.520661,01073,Jefferson,010730027001097
3,5,5,5,456283013907628034,2014-04-16 00:08:42,http://twitter.com/maya32603/status/4562830139...,RT @TehachapiHomes Harry Reid on Bundy Ranch: ...,@maya32603,MaryW,United States of America,...,NaN,NaN,NaN,NaN,NaN,-122.066635,38.257725,06095,Solano,060952523052012
4,7,7,7,508260236432801792,2014-09-06 10:27:38,http://twitter.com/KylaCampbellDC/status/50826...,MT @NBCNightlyNews: DETAILS: Pres #Obama to de...,@KylaCampbellDC,Kyla Campbell,United States of America,...,NaN,NaN,NaN,NaN,NaN,-77.036871,38.907192,11001,District of Columbia,110010053013007


In [9]:
dtype_dic= {'FIPS': str, 
            'FIPS_block' : str}

indicators = pd.read_csv('/Users/rorr/PythonStuff/Team_5/Resources/Census Resources/indicators.csv'
                         ,dtype = dtype_dic
                        )

indicators=indicators.drop(columns=['Unnamed: 0','Unnamed: 0.1'])

#print(indicators.head(2))
for (idx, row) in indicators.iterrows():
    fips_z = (row.loc['FIPS'])
    indicators.at[idx, 'FIPS'] = str(fips_z).zfill(5)

census = pd.read_csv('/Users/rorr/PythonStuff/Team_5/Resources/Census Resources/census_immig.csv'
                         ,dtype = dtype_dic, engine='python'                    )
census=census.drop(columns=['Unnamed: 0','index'])

for (idx, row) in census.iterrows():
    fips_z = (row.loc['combined_fips'])
    census.at[idx, 'FIPS'] = str(fips_z).zfill(5)
    
merged_tweet1 = pd.merge(immigration_df, census, on=["FIPS"], how="outer")
merged_tweet1 = pd.merge(merged_tweet1, indicators, on=["FIPS","County Name",'Household Income','Population',
                                                        'ForeignBorn_Rate','Per Capita Income',
                                                        'state_abbr','Poverty Rate'], how="outer")

print(merged_tweet1.head(3))

   index  Unnamed: 0  Unnamed: 0.1          GUID           Date (EST)  \
0    0.0         0.0           0.0  4.586112e+17  2014-04-22 10:20:00   
1   42.0        42.0          42.0  4.239127e+17  2014-01-16 15:20:43   
2   44.0        44.0          44.0  4.366754e+17  2014-02-20 20:34:58   

                                                 URL  \
0  http://twitter.com/aretana/status/458611179154...   
1  http://twitter.com/ReFlex76/status/42391272196...   
2  http://twitter.com/LAimigrationLaw/status/4366...   

                                            Contents            Author  \
0  RT @NateSilver538 Contrary to stereotypes, man...          @aretana   
1  RT @Green_Footballs Anti-Immigrant Group With ...         @ReFlex76   
2  Please see my answer on @Avvo to: Can I travel...  @LAimigrationLaw   

                  Name                   Country  \
0       Alberto Retana  United States of America   
1  Antonio E. Gonzalez  United States of America   
2  Aggie R Hoffman Law  Unite

In [7]:
colorscale = ["#fff7ec","#fee8c8","#fdd49e","#fdbb84","#fc8d59","#ef6548","#d7301f","#b30000","#7f0000"]
endpts = list(np.linspace(0, 40, len(colorscale) - 1))
fips = merged_tweet2['FIPS']
values = (merged_tweet2['Contents'])

#import plotly.figure_factory as ff  
#import plotly.figure_factory as ff
#import plotly

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['usa'],
    binning_endpoints=endpts,
    colorscale=colorscale,
    #county_outline={'color': 'rgb(15, 15, 15)', 'width': 0.5},
    state_outline={'color': 'rgb(200, 175, 150)','width': 1},
    show_state_data=True,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9, title='Racial Anxiety by County',
    legend_title='Racial Anxiety Raw')
fig
py.plot( fig, filename='Raw_Racism-map.html' )

'file:///Users/rorr/PythonStuff/Team_5/Raw_Racism-map.html'

In [ ]:
# # Run Vader Analysis on each tweet
# for (idx, row) in output_list.iterrows():
#     text = (row.loc['text_tweet'])
#     counter += 1
#     results = analyzer.polarity_scores(text)
#     compound = results["compound"]
#     pos = results["pos"]
#     neu = results["neu"]
#     neg = results["neg"]
#     tweet_times.append(tweet["created_at"])
#     output_list.at[idx, 'compound'] = compound
#     output_list.at[idx, 'pos'] = pos
#     output_list.at[idx, 'neu'] = neu
#     output_list.at[idx, 'neg'] = neg
